# SQL and Parquet

- Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

- For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

- The `spark` object and the `AA_DFW_ALL.parquet` file are available for you automatically.

## Instructions

- Import the `AA_DFW_ALL.parquet` file into `flights_df`.
- Use the `createOrReplaceTempView` method to alias the `flights` table.
- Run a Spark SQL query against the `flights` table.

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [11]:
# Read the Parquet file into flights_df
flights_df = spark.read.load().parquet('file:///home/talentum/AA_DFW_ALL.parquet')
flights_df = flights_df.withColumn("flight_duration", flights_df.flight_duration.cast('double'))

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights')
# print('The average flight time is: %f' % avg_duration)

In [12]:
flights_df.describe()

DataFrame[summary: string, _c0: string, _c1: string, _c2: string, flight_duration: string]

In [13]:
flights_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- flight_duration: double (nullable = true)



In [14]:
avg_duration.printSchema()

root
 |-- avg(flight_duration): double (nullable = true)



In [15]:
count_duration.printSchema()

root
 |-- count: long (nullable = false)



In [16]:
count_duration.toPandas()

,count
0,259268


In [28]:
x=avg_duration.toPandas()

In [30]:
x.iloc[0][0]

151.68865806809941